<a href="https://colab.research.google.com/github/davidnoone/ENVPHYS300/blob/main/ENVPHYS300_Lab1_part2_RCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ENVPHYS300: Lab 1 (part 2)


The radiative balance of the planet depends on the partitioning of energy within the atmosphere.

Building on the analysis of surface energy balance, we will examine which processes and characteristics influence the energy balance. Specifically, we know that solar radiation passes through the atmosphere to warm the Earth's surface. Surface emission of radiation and turbulent heat fluxes allow energy to move upward into the boundary layer. Ultimately, this energy leads to thermal instability that creates buoyant plumes of air. These plumes can transport heat upward via (dry) turbulence or lead to the onset of condensation and possibly convective cloud formation, efficiently heating the atmosphere through latent heat release.

There is a complementary balance between the radiative budget and convective activity. This defines the concept of radiative-convective equilibrium. The details of that equilibrium state depend on the composition of the atmosphere, its radiative properties, and surface characteristics.

Goal: This lab consists of tasks aimed at developing numerical experiments to evaluate how the atmospheric temperature profile and energy balance depend on these driving factors.


###A Radiative-Convective Equilibrium Model
We will use a model to simulate radiative-convective equilibrium. Specifically, we will use the KONRAD model, which has been employed as a research tool to evaluate climate and climate sensitivity



Details of the model can be found here: https://konrad.readthedocs.io/index.html

Helpful examples can be found here: https://github.com/atmtools/konrad/tree/main/howto

The first test below is described in the "getting started" document:
https://github.com/atmtools/konrad/blob/main/howto/getting_started.md


Some research papers using this model:

Sally Dacie, Lukas Kluft, Hauke Schmidt, Bjorn Stevens, Stefan A. Buehler, Peer J. Nowack, Simone Dietmüller, N. Luke Abraham, and Thomas Birner. A 1d rce study of factors affecting the tropical tropopause layer and surface climate. Journal of Climate, 32(20):6769 – 6782, 2019. doi:10.1175/JCLI-D-18-0778.1.

Lukas Kluft, Sally Dacie, Manfred Brath, Stefan A. Buehler, and Bjorn Stevens. Temperature-dependence of the clear-sky feedback in radiative-convective equilibrium. Geophysical Research Letters, 48(22):e2021GL094649, 2021. doi:https://doi.org/10.1029/2021GL094649.

Lukas Kluft, Sally Dacie, Stefan A. Buehler, Hauke Schmidt, and Bjorn Stevens. Re-examining the first climate models: climate sensitivity of a modern radiative–convective equilibrium model. Journal of Climate, 32(23):8111–8125, 2019. doi:10.1175/JCLI-D-18-0774.1.

In [ ]:
!pip install konrad

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import konrad

#Tasl 0: Running the model

The model siulates the radiative transfer of energy, with absoption and emission in both longwave and infrared parts of the spectrum, as well as the generation of moist convection. This is performed for a single column of the atmosphere. This can be configured to represent the global average, or some particular region on earth.

To run the model, we must established the pressure range of which calculations are to be performed. Since we need to compute the full atmosphere, we will use a "top of the atmosphere (TOA) to be 1 Pa!


In [ ]:

# Set model fromain from 1000 to 0.01 hPa!
plev, phlev = konrad.utils.get_pressure_grids(1000e2, 1, 128)

# Define the atmosphere - using all defaults here
atmosphere_std = konrad.atmosphere.Atmosphere(phlev)


In [ ]:
# Example plotting function to plot the temperature profile
def plot_temperature_profile(pressure,temperature, overlay=False,title=None,label=None,
                             p_min=10,p_max=1000):
    tickvals = [1000, 500, 250, 100, 50, 25, 10]      # tick marks in hPa
    ticklabels = [str(t) for t in tickvals]                 # Convert to string for labels

    # Use overlay if you wish to add extra temperature profiles
    if not overlay:
        fig, ax = plt.subplots()
        ax.set_xlabel(r"Temperature [K]")

        ax.set_ylabel("Pressure [hPa]")
        ax.set_yscale('log')            # log scale for pressure
        ax.set_ylim(p_max, p_min)           # Reverse from high to low
        ax.set_yticks(tickvals)
        ax.set_yticklabels(ticklabels)

        ax.legend()
        if title is not None:
            ax.set_title(title)

    plt.plot(temperature, pressure/100, label=label)        # notice conversion dPa->hPa
    return




In [ ]:
#
# Test plot of the default temperature profile
#
plot_temperature_profile(atmosphere_std['plev'], atmosphere_std['T'][-1, :],title='Standard Atmosphere')

##Radiative convective equilibrium experiment

Let us configure a model simulation that has a fixed surface temperatre.






In [ ]:
# Initialize the setup for the radiative-convective equilibrium simulation.
atmosphere = atmosphere_std
rce_test = konrad.RCE(
    atmosphere,
    surface=konrad.surface.FixedTemperature(temperature=288.),  # Run with a fixed surface temperature.
    timestep='12h',  # Set timestep in model time.
    max_duration='100d',  # Set maximum runtime.
)

##Running the model

Once configured, we need to run the model. The model runs with small (12 hour) steps in time. Each time step the radiative transfer calculation is performed, and a test is done to check if there is instabilit yto trigger convection. If so, and algorithm to model convective updrafts and condensation is performed, and temperature and humidity of the profile are updated.

After any time steps, and equilibrium (or, quasi equilibrium) is found.

In [ ]:
rce_test.run()  # Start the simulation

In [ ]:
#
# Plot the simulated equikubrium state overlayed wit the original "standard atmosphere"
#
plot_temperature_profile(atmosphere_std['plev'], atmosphere_std['T'][-1, :],label='Standard Atmosphere',title='Default experiment')
plot_temperature_profile(rce_test.atmosphere['plev'], rce_test.atmosphere['T'][-1, :],label='RCE test profile',overlay=True)


In the example above, notice that the surface temperature was set as a boundary condition. Thus, it is not particularly reasonable to use the predicted surface temperature as a measningful result. We set it!

Let's reconfigure the model to have an ocean that allows for the surface temperature to be computed. The "ocean" will be 50m deep, and used as a heat reservoir that can come into thermal equilibrium.

From here on, we will use this slab ocean configuration as our default ("control") experiment. Notice, a "control" is an important part of any scientific experiment - which is true in a physical lab, and equally true in a digital lab.



In [ ]:
#
# Define a slab ocean model, an initialize the model with e
#
print("Top of the atmosphere energy flux: ",rce_test.radiation["toa"][-1])

slab = konrad.surface.SlabOcean(
    temperature=rce_test.surface["temperature"][-1],
    heat_sink=rce_test.radiation["toa"][-1],
)

print("Initial temperature: ", slab["temperature"][0])

#
# reconfigure the model: create an instance that we can save as the control data.
#
rce_control = konrad.RCE(
    atmosphere,
    surface=slab,  # Run with a slab surface.
    timestep='12h',  # Set timestep in model time.
    max_duration='300d',  # Set maximum runtime.
)
print('Running the model fo 300 days... this will take a moment...')
rce_control.run()  # Start the simulation.

print("Temperature after 300 days: ", slab["temperature"][0])


In [ ]:
# Double check that they are similar!
#plot_temperature_profile(rce_test.atmosphere['plev'], rce_test.atmosphere['T'][-1, :],label='RCE test profile')
#plot_temperature_profile(rce_control.atmosphere['plev'], rce_control.atmosphere['T'][-1, :],label='RCE slab ocean',overlay=True)


## TASK 1: Compute a radiative equlibrium profile
Above, the default model provides a simulation that balances radiative heating and convective processes. Configure the model to prduce a radiative equilibrium profile. That is, excluding convection. Add this predicted profile to the plot.


Hint: Check the how-to document for how to turn off convection when you define the model.
https://github.com/atmtools/konrad/blob/main/howto/convection.md



---


a. Describe in words how the radiative profile differs from the RCE profile.

b. Do you expect the radiative equilibrium profile to be stable or unstable with respective to convective motions. [Hint: you may wish to find the lapse rate for the troposphere, and compare this to a dry and expected moist adibatic lapse rate]

c. How does the radiatie equilibrium profile relate to the top of the atmsphere radiative equilibrium temperature that we computed in class, when we considerd the earth with no atmosphere.



---







In [ ]:
# [your code here]

[Your text answer/discussion here]

a.

b.

c.

## Task 2: Dependence on atmospheric composition

The greenhouse effect arrises from the trapping of infrared by various gases. We wish to evalue this effect, and evalute the importance of both ozone and CO2. Additionally, we wish to evalue the importance of water vapor in providing a feedback that might amplify the effect of direct absorption


## Ozone influence on radiative equilibrium profile
a. Evaluate the dependence of the temperature profile on ozone. Re-run the model setting the ozone ("O3") concentration to zero. Plot the temperature and humidity profiles from this experiment and from te control to show the contrast. Report the surface temperature. Describe the differences, and provide an explaination (short) of your result.


Hint: You can set the profile of different gases via the "atmosphere" object. Consider the following:


> atmosphere['O3'] = ozone_profile



In [ ]:
#
# [your ozone experiment an plots here]
#

## CO2 dependene

b. Evaluate the influence of carbon dioxide (CO2). Starting again from the control experiment (i.e., make sure you keep the ozone!), re run the model for the case for double the CO2 concentation. Again report the surface temperature, and describe the changes to the profile of temperature and humidity.

Hint: As for ozone, you can set the CO2 concentration as:

> atmosphere['CO2'] = your_desired value





In [ ]:
#
# [your CO2 experiment and plots here]
#
#

## Water vapor

c. In both cases, you might find that the water vapor concentration has changed. Let us test of the water vapor plays a role in changeing the temperature. Perform the CO2 doubling experiment again, but in this case also keep the H2O concentration the same as in the control case. This way we can evaluate the warming due to CO2 radiative influence only.

[Hint: This is a little more involved for the set up. You will need to save the humidity profile from your control experiment, and then prescribe this as a fixed concentration (volume mixing ratio, VMR) in the new experiment.


> atmosphere["H2O"][:] = profile_from_previous_experiment

> humidity_model = konrad.humidity.FixedVMR()  # Preserve the absolute humidity


This can then be used in the way you configure the model.

> rce = (..., humidity=humidity_model, ...)

Also see https://github.com/atmtools/konrad/blob/main/howto/humidity.md







---


Comment on the differences in the surface temperature and the modeled temperature profile. Contrast this with the profile you have from your first CO2-doubling experiment. Descrine in what ways that water vapor (and the hydrological cycle as a whole) might be considered to provide a feedback.



In [ ]:
#
# [your CO2 experiment, with fixed H2O, and plots here]
#




#Water vapor feedbacks

d. What what magniture of warming there is with and without the water vapor feedback. What percentage of the total warming at the surface is due to the water vapor feedback.

[Your answer here]